<div align="center">
<img src = "Logo-OC.png", width = 300>
</div>

<font color=blue>
<h2>PROJET 3 - RÉALISER UNE ÉTUDE DE SANTÉ PUBLIQUE - Mission 3<a class="tocSkip"></h2>
    <h3>David Raux - 2018 <a class="tocSkip"></h3>
</font>
<font color=blue>
    
https://openclassrooms.com/fr/projects/realisez-une-etude-de-sante-publique
    
</font>

***
Lien pour télécharger les données : http://www.fao.org/faostat/fr/#data/FBS
***

### Sommaire :<a id="sommaire"></a>

> [Partie 1 - Importation des fichiers](#partie1)
> 
> [Partie 2 - Création des tables](#partie2)
> - [Table 1 - population](#2.1)
> - [Table 2 - disponibilité alimentaire](#2.2)
> - [Table 3 - equilibre_prod](#2.3)
> - [Table 4 - sous_nutrition](#2.4)
> - [Création des tables en SQL](#2.5)
>
> [Partie 3 - Question 19 - Requêtes SQL](#partie3)
> - [R1. Les 10 pays avec le plus **fort** ratio de disponibilité alimentaire par habitant par jour (en protéine et Kcal)](#3.1)
> - [R2. Les 10 pays avec le plus **faible** ratio de disponibilité alimentaire par habitant par jour (en protéine et Kcal)](#3.2)
> - [R3. Les 10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte.](#3.3)
> - [R4. Les 10 produits pour lesquels le ratio Autres utilisations / Disponibilité intérieure est le plus élevé.](#3.4)
> - [Q20. Analyse des autres utilisations non alimentaires possibles](#3.5)
>

In [1]:
import pandas as pd
import os
import numpy as np
from functools import reduce
import sqlite3 as sql

In [2]:
os.getcwd()

'/Users/davidraux/Google Drive/Parcours OC Data analyst/P3 - Réaliser une étude de santé publique'

***
# <font color=orange> Partie 1 - Importer les fichiers </font><a id='partie1'> </a>
---

In [3]:
population_brut = pd.read_csv('Donnees/mission3/population_FAO.csv')
animaux_brut = pd.read_csv('Donnees/mission3/animaux_FAO.csv')
vegetaux_brut = pd.read_csv('Donnees/mission3/vegetaux_FAO.csv')
equilibre_prod_brut = pd.read_csv('Donnees/mission3/equilibre_prod_FAO.csv')
sous_nutrition_brut = pd.read_csv('Donnees/mission3/sous_nutrition_FAO.csv')

[Retour au sommaire](#sommaire)

***
# <font color=orange> Partie 2 - Création des tables </font><a id='partie2'> </a>
---

## <font color=blue>Table 1 - population<a id='2.1'> </a>

In [4]:
## Sélectionner et renommer les colonnes
population = population_brut[['Pays','Code Pays', 'Année', 'Valeur']]
population.columns = ['pays', 'code_pays', 'année', 'population']
population['population'] *= 10**3
population.head()

/Users/davidraux/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,pays,code_pays,année,population
0,Afghanistan,2,2010,28398000
1,Afghanistan,2,2011,29105000
2,Afghanistan,2,2012,29825000
3,Afghanistan,2,2013,30552000
4,Afrique du Sud,202,2010,51452000


In [5]:
## Exportation en fichier .csv
population.to_csv('Donnees/mission3/population.csv', index = False)

[Retour au sommaire](#sommaire)

## <font color=blue>Table 2 - dispo_alim<a id='2.2'> </a>

In [6]:
## Regrouper les 2 fichiers et apporter l'information de la nature du produit
animaux = animaux_brut.assign(nature_produit = 'animaux')
vegetaux = vegetaux_brut.assign(nature_produit = 'vegetaux')
dispo_alim_total = pd.concat([vegetaux, animaux])
dispo_alim_total = dispo_alim_total[['Pays','Code Pays','Année','Élément','Code Élément',
                                     'Produit','Code Produit','Valeur','nature_produit']]
dispo_alim_total.head()

,Pays,Code Pays,Année,Élément,Code Élément,Produit,Code Produit,Valeur,nature_produit
0,Afghanistan,2,2010,Disponibilité alimentaire en quantité (kg/pers...,645,Blé,2511,162.16,vegetaux
1,Afghanistan,2,2011,Disponibilité alimentaire en quantité (kg/pers...,645,Blé,2511,161.85,vegetaux
2,Afghanistan,2,2012,Disponibilité alimentaire en quantité (kg/pers...,645,Blé,2511,161.27,vegetaux
3,Afghanistan,2,2013,Disponibilité alimentaire en quantité (kg/pers...,645,Blé,2511,160.23,vegetaux
4,Afghanistan,2,2010,Disponibilité alimentaire (Kcal/personne/jour),664,Blé,2511,1386.00,vegetaux


In [7]:
## Création du dataframe'dispo_alim_kg'
dispo_alim_kg = dispo_alim_total.loc[dispo_alim_total['Code Élément']==645]
dispo_alim_kg = dispo_alim_kg[['Pays','Code Pays','Année','Produit','Code Produit',
                                              'Valeur','nature_produit']]
dispo_alim_kg.columns = ['pays', 'code_pays', 'année', 'produit', 'code_produit', 
                                'dispo_alim_tonnes','nature_produit']

## Création du dataframe'dispo_alim_kcal'
dispo_alim_kcal = dispo_alim_total.loc[dispo_alim_total['Code Élément']==664]
dispo_alim_kcal = dispo_alim_kcal[['Pays','Code Pays','Année','Produit','Code Produit',
                                              'Valeur','nature_produit']]
dispo_alim_kcal.columns = ['pays', 'code_pays', 'année', 'produit', 'code_produit', 
                                'dispo_alim_kcal_p_j','nature_produit']

## Création du dataframe'dispo_prot'
dispo_prot = dispo_alim_total.loc[dispo_alim_total['Code Élément']==674]
dispo_prot = dispo_prot[['Pays','Code Pays','Année','Produit','Code Produit',
                                              'Valeur','nature_produit']]
dispo_prot.columns = ['pays', 'code_pays', 'année', 'produit', 'code_produit', 
                                'dispo_prot','nature_produit']

## Création du dataframe'dispo_mat_gr'
dispo_mat_gr = dispo_alim_total.loc[dispo_alim_total['Code Élément']==684]
dispo_mat_gr = dispo_mat_gr[['Pays','Code Pays','Année','Produit','Code Produit',
                                              'Valeur','nature_produit']]
dispo_mat_gr.columns = ['pays', 'code_pays', 'année', 'produit', 'code_produit', 
                                'dispo_mat_gr','nature_produit']

In [8]:
## Jointure des 4 dataframes
dispo_alim_1 = pd.merge(dispo_alim_kg,dispo_alim_kcal, how='outer')
dispo_alim_2 = pd.merge(dispo_alim_1,dispo_prot, how='outer')
dispo_alim = pd.merge(dispo_alim_2,dispo_mat_gr, how='outer')

## Remplacer les Na par des 0
dispo_alim.fillna(value=0, inplace=True)
dispo_alim.head()

,pays,code_pays,année,produit,code_produit,dispo_alim_tonnes,nature_produit,dispo_alim_kcal_p_j,dispo_prot,dispo_mat_gr
0,Afghanistan,2,2010,Blé,2511,162.16,vegetaux,1386.0,37.32,5.01
1,Afghanistan,2,2011,Blé,2511,161.85,vegetaux,1383.0,37.29,4.73
2,Afghanistan,2,2012,Blé,2511,161.27,vegetaux,1378.0,37.16,4.64
3,Afghanistan,2,2013,Blé,2511,160.23,vegetaux,1369.0,36.91,4.69
4,Afghanistan,2,2010,Riz (Eq Blanchi),2805,15.58,vegetaux,159.0,3.04,0.30


In [9]:
## Exportation en fichier .csv
dispo_alim.to_csv('Donnees/mission3/dispo_alim.csv', index = False)

[Retour au sommaire](#sommaire)

## <font color=blue>Table 3 - equilibre_prod<a id='2.3'> </a>

In [10]:
equilibre_prod_brut['Élément'].unique()

array(['Disponibilité intérieure', 'Semences', 'Pertes', 'Nourriture',
       'Aliments pour animaux', 'Traitement', 'Autres Utilisations'],
      dtype=object)

In [11]:
equilibre_prod_pivot = pd.pivot_table(equilibre_prod_brut, values ='Valeur',
                                index=['Pays','Code Pays','Année','Produit','Code Produit'],
                                columns=['Élément'],
                                aggfunc=np.sum)
equilibre_prod_pivot.fillna(value = 0, inplace=True)
equilibre_prod_pivot.reset_index()
equilibre_prod_pivot.head()

Élément                                                         Aliments pour animaux  \
Pays        Code Pays Année Produit               Code Produit                          
Afghanistan 2         2010  Abats Comestible      2736                            0.0   
                            Agrumes, Autres       2614                            0.0   
                            Aliments pour enfants 2680                            0.0   
                            Ananas                2618                            0.0   
                            Bananes               2615                            0.0   

Élément                                                         Autres Utilisations  \
Pays        Code Pays Année Produit               Code Produit                        
Afghanistan 2         2010  Abats Comestible      2736                          0.0   
                            Agrumes, Autres       2614                          0.0   
                            Aliments pour enfants 2680                          0.0   
                            Ananas                2618                          0.0   
                            Bananes               2615                          0.0   

Élément                                                         Disponibilité intérieure  \
Pays        Code Pays Année Produit               Code Produit                             
Afghanistan 2         2010  Abats Comestible      2736                              54.0   
                            Agrumes, Autres       2614                               3.0   
                            Aliments pour enfants 2680                               2.0   
                            Ananas                2618                               0.0   
                            Bananes               2615                              21.0   

Élément                                                         Nourriture  \
Pays        Code Pays Année Produit               Code Produit               
Afghanistan 2         2010  Abats Comestible      2736                54.0   
                            Agrumes, Autres       2614                 3.0   
                            Aliments pour enfants 2680                 2.0   
                            Ananas                2618                 0.0   
                            Bananes               2615                21.0   

Élément                                                         Pertes  \
Pays        Code Pays Année Produit               Code Produit           
Afghanistan 2         2010  Abats Comestible      2736             0.0   
                            Agrumes, Autres       2614             0.0   
                            Aliments pour enfants 2680             0.0   
                            Ananas                2618             0.0   
                            Bananes               2615             0.0   

Élément                                                         Semences  \
Pays        Code Pays Année Produit               Code Produit             
Afghanistan 2         2010  Abats Comestible      2736               0.0   
                            Agrumes, Autres       2614               0.0   
                            Aliments pour enfants 2680               0.0   
                            Ananas                2618               0.0   
                            Bananes               2615               0.0   

Élément                                                         Traitement  
Pays        Code Pays Année Produit               Code Produit              
Afghanistan 2         2010  Abats Comestible      2736                 0.0  
                            Agrumes, Autres       2614                 0.0  
                            Aliments pour enfants 2680                 0.0  
                            Ananas                2618                 0.0  
                            Bananes               2615      

In [12]:
## Création du dataframe'aliments_animaux'
alim_ani = equilibre_prod_brut.loc[equilibre_prod_brut['Code Élément']==5521]
alim_ani = alim_ani[['Pays','Code Pays','Année','Produit','Code Produit','Valeur']]
alim_ani.columns = ['pays', 'code_pays', 'année','produit', 'code_produit','alim_ani']

## Création du dataframe'semences'
semences = equilibre_prod_brut.loc[equilibre_prod_brut['Code Élément']==5527]
semences = semences[['Pays','Code Pays','Année','Produit','Code Produit','Valeur']]
semences.columns = ['pays', 'code_pays', 'année','produit', 'code_produit','semences']

## Création du dataframe'pertes'
pertes = equilibre_prod_brut.loc[equilibre_prod_brut['Code Élément']==5123]
pertes = pertes[['Pays','Code Pays','Année','Produit','Code Produit','Valeur']]
pertes.columns = ['pays', 'code_pays', 'année','produit', 'code_produit','pertes']

## Création du dataframe'transfo'
transfo = equilibre_prod_brut.loc[equilibre_prod_brut['Code Élément']==5131]
transfo = transfo[['Pays','Code Pays','Année','Produit','Code Produit','Valeur']]
transfo.columns = ['pays', 'code_pays', 'année','produit', 'code_produit','transfo']

## Création du dataframe'nourriture'
nourriture = equilibre_prod_brut.loc[equilibre_prod_brut['Code Élément']==5142]
nourriture = nourriture[['Pays','Code Pays','Année','Produit','Code Produit','Valeur']]
nourriture.columns = ['pays', 'code_pays', 'année','produit', 'code_produit','nourriture']

## Création du dataframe'autres_utilisations'
autres_utilisations = equilibre_prod_brut.loc[equilibre_prod_brut['Code Élément']==5154]
autres_utilisations = autres_utilisations[['Pays','Code Pays','Année','Produit','Code Produit','Valeur']]
autres_utilisations.columns = ['pays', 'code_pays', 'année','produit', 'code_produit','autres_utilisations']

## Création du dataframe'dispo_int'
dispo_int = equilibre_prod_brut.loc[equilibre_prod_brut['Code Élément']==5301]
dispo_int = dispo_int[['Pays','Code Pays','Année','Produit','Code Produit','Valeur']]
dispo_int.columns = ['pays', 'code_pays', 'année','produit', 'code_produit','dispo_int']

In [13]:
## Jointure des 7 tables
equilibre_prod_dfs = [alim_ani,semences,pertes,transfo,nourriture,autres_utilisations,dispo_int]

equilibre_prod = reduce(lambda left,right: pd.merge(left,right, how='outer', 
                                            on=['pays','code_pays','produit', 'année','code_produit']),
                         equilibre_prod_dfs)

equilibre_prod = equilibre_prod[['pays', 'code_pays', 'année', 'produit', 'code_produit', 
                                   'dispo_int', 'alim_ani', 'semences', 'pertes', 'transfo', 
                                   'nourriture', 'autres_utilisations']]
equilibre_prod.fillna(value = 0, inplace=True)
equilibre_prod.head()

,pays,code_pays,année,produit,code_produit,dispo_int,alim_ani,semences,pertes,transfo,nourriture,autres_utilisations
0,Afghanistan,2,2010,Orge,2513,440,315.0,21.0,44.0,0.0,60.0,0.0
1,Afghanistan,2,2011,Orge,2513,339,203.0,31.0,34.0,0.0,72.0,0.0
2,Afghanistan,2,2012,Orge,2513,519,367.0,22.0,52.0,0.0,78.0,0.0
3,Afghanistan,2,2013,Orge,2513,524,360.0,22.0,52.0,0.0,89.0,0.0
4,Afghanistan,2,2010,Maïs,2514,301,178.0,11.0,30.0,0.0,82.0,0.0


In [14]:
## Exportation en fichier .csv
equilibre_prod.to_csv('Donnees/mission3/equilibre_prod.csv', index = False)

[Retour au sommaire](#sommaire)

## <font color=blue>Table 4 - sous_nutrition<a id='2.4'> </a>

In [15]:
## Création du dataframe'sous_nutrition'
sous_nutrition = sous_nutrition_brut[['Zone','Code zone','Année','Valeur']]
sous_nutrition.columns = ['pays', 'code_pays', 'année','nb_personnes']
sous_nutrition['année'].replace(['2009-2011', '2010-2012', '2011-2013', '2012-2014', '2013-2015', '2014-2016'],
                                [2010, 2011, 2012, 2013, 2014, 2015], inplace=True)
sous_nutrition['nb_personnes'] *= 10**6 # Pour ne pas les avoir en millions
sous_nutrition.head()

/Users/davidraux/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/Users/davidraux/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,pays,code_pays,année,nb_personnes
0,Afghanistan,2,2010,6200000.0
1,Afghanistan,2,2011,6400000.0
2,Afghanistan,2,2012,6800000.0
3,Afghanistan,2,2013,7200000.0
4,Afghanistan,2,2014,7500000.0


In [16]:
## Exportation en fichier .csv
sous_nutrition.to_csv('Donnees/mission3/sous_nutrition.csv', index = False)

[Retour au sommaire](#sommaire)

# <font color=blue>Création des tables en SQL<a id='2.5'> </a>

In [17]:
db = sql.connect('Donnees/mission3/mission3_SQLite3_Jupyter.db')

In [18]:
# Création de la table 'population'
cursor = db.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS population(
                pays TEXT NOT NULL, 
                code_pays INTEGER NOT NULL, 
                année REAL NOT NULL, 
                population INTEGER,
                PRIMARY KEY(année, code_pays))
                ''')

# Création de la table 'dispo_alim'
cursor.execute('''CREATE TABLE IF NOT EXISTS dispo_alim(
                pays TEXT NOT NULL, 
                code_pays INTEGER NOT NULL, 
                année REAL NOT NULL, 
                produit TEXT NOT NULL,
                code_produit INTEGER NOT NULL,
                dispo_alim_tonnes REAL,
                nature_produit TEXT NOT NULL,
                dispo_alim_kcal_p_j REAL,
                dispo_prot FLOAT, 
                dispo_mat_gr FLOAT, 
                PRIMARY KEY(année, code_pays, produit))
                ''')

# Création de la table 'equilibre_prod'
cursor.execute('''CREATE TABLE IF NOT EXISTS equilibre_prod(
                pays TEXT NOT NULL, 
                code_pays INTEGER NOT NULL, 
                année REAL NOT NULL, 
                produit TEXT NOT NULL,
                code_produit INTEGER NOT NULL, 
                alim_ani REAL, 
                semences REAL,
                pertes REAL, 
                transfo REAL, 
                nourriture REAL,
                autres_utilisations REAL,
                dispo_int REAL,
                PRIMARY KEY(année,code_pays,produit))
                ''')

# Création de la table 'sous_nutrition'
cursor.execute('''CREATE TABLE IF NOT EXISTS sous_nutrition(
                pays TEXT NOT NULL, 
                code_pays INTEGER NOT NULL, 
                année REAL NOT NULL, 
                nb_personnes REAL,
                PRIMARY KEY(année,code_pays))
                ''')
db.commit()

In [19]:
## On remplit les tables SQL avec les fichiers .csv créés précédemment
population.to_sql('population', con=db, if_exists='replace')
dispo_alim.to_sql('dispo_alim', con=db, if_exists='replace')
equilibre_prod.to_sql('equilibre_prod', con=db, if_exists='replace')
sous_nutrition.to_sql('sous_nutrition', con=db, if_exists='replace')

***
# <font color=orange> Partie 3 - Requêtes SQL </font><a id='partie3'> </a>
---

[Retour au sommaire](#sommaire)

# <font color=blue>Requête 1 <a id='3.1'> </a>
    Les 10 pays avec la plus forte disponibilité alimentaire par habitant par jour (en protéine et Kcal)

In [29]:
Q1_g_prot = pd.read_sql_query('''
SELECT pays, sum(dispo_prot) AS dispo_alim_prot_g_jour_habitant
FROM dispo_alim
WHERE année = 2013
GROUP BY pays
ORDER BY dispo_alim_prot_g_jour_habitant DESC
LIMIT 10;
''', db)

# On effectue la requête SQL pour avoir le ratio en Kcal
Q1_kg_prot_kcal = pd.read_sql_query('''
SELECT pays, (sum(dispo_alim_kcal_p_j)) AS dispo_alim_kcal_jour_habitant
FROM dispo_alim
WHERE année = 2013
GROUP BY pays
ORDER BY dispo_alim_kcal_jour_habitant DESC
LIMIT 10;
''', db)

# On affiche les résultats
print('-'*100)
print('10 pays ayant le plus haut ratio "disponibilité alimentaire/habitant" en terme de protéines (en g):')
display(Q1_g_prot)
print('-'*100)
print('10 pays ayant le plus haut ratio "disponibilité alimentaire/habitant" en terme de calories (en Kcal):')
display(Q1_kg_prot_kcal)

----------------------------------------------------------------------------------------------------
10 pays ayant le plus haut ratio "disponibilité alimentaire/habitant" en terme de protéines (en g):


,pays,dispo_alim_prot_g_jour_habitant
0,Islande,133.06
1,Chine - RAS de Hong-Kong,129.07
2,Israël,128.00
3,Lituanie,124.36
4,Maldives,122.32
5,Finlande,117.56
6,Luxembourg,113.64
7,Monténégro,111.90
8,Pays-Bas,111.46
9,Albanie,111.37


----------------------------------------------------------------------------------------------------
10 pays ayant le plus haut ratio "disponibilité alimentaire/habitant" en terme de calories (en Kcal):


,pays,dispo_alim_kcal_jour_habitant
0,Autriche,3770.0
1,Belgique,3737.0
2,Turquie,3708.0
3,États-Unis d'Amérique,3682.0
4,Israël,3610.0
5,Irlande,3602.0
6,Italie,3578.0
7,Luxembourg,3540.0
8,Égypte,3518.0
9,Allemagne,3503.0


[Retour au sommaire](#sommaire)

# <font color=blue>Requête 2 <a id='3.2'> </a>
    10 pays avec la plus faible disponibilité alimentaire par habitant par jour (en protéine et Kcal)

In [31]:
## Requête pour avoir le ratio en protéine
R2_prot = pd.read_sql_query('''
SELECT pays, sum(dispo_prot) AS dispo_alim_prot_g_jour_habitant
FROM dispo_alim
WHERE année = 2013
GROUP BY pays
ORDER BY dispo_alim_prot_g_jour_habitant
LIMIT 10;
''', db)

## Requête pour avoir le ratio en Kcal
R2_kcal = pd.read_sql_query('''
SELECT pays, sum(dispo_alim_kcal_p_j) AS dispo_alim_kcal_jour_habitant
FROM dispo_alim
WHERE année = 2013
GROUP BY pays
ORDER BY dispo_alim_kcal_jour_habitant
LIMIT 10;
''', db)

## Affichage des résultats
print('-'*100)
print('10 pays ayant le plus faible ratio "disponibilité alimentaire/habitant" en terme de protéines (en g):')
display(R2_prot)
print('-'*100)
print('10 pays ayant le plus faible ratio "disponibilité alimentaire/habitant" en terme de calories (en Kcal):')
display(R2_kcal)

----------------------------------------------------------------------------------------------------
10 pays ayant le plus faible ratio "disponibilité alimentaire/habitant" en terme de protéines (en g):


,pays,dispo_alim_prot_g_jour_habitant
0,Libéria,37.66
1,Guinée-Bissau,44.05
2,Mozambique,45.68
3,République centrafricaine,46.04
4,Madagascar,46.69
5,Haïti,47.70
6,Zimbabwe,48.32
7,Congo,51.41
8,Ouganda,52.64
9,Sao Tomé-et-Principe,53.10


----------------------------------------------------------------------------------------------------
10 pays ayant le plus faible ratio "disponibilité alimentaire/habitant" en terme de calories (en Kcal):


,pays,dispo_alim_kcal_jour_habitant
0,République centrafricaine,1879.0
1,Zambie,1924.0
2,Madagascar,2056.0
3,Afghanistan,2087.0
4,Haïti,2089.0
5,République populaire démocratique de Corée,2093.0
6,Tchad,2109.0
7,Zimbabwe,2113.0
8,Ouganda,2126.0
9,Timor-Leste,2129.0


[Retour au sommaire](#sommaire)

# <font color=blue>Requête 3 <a id='3.3'> </a>
    10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte.

In [36]:
R3 = pd.read_sql_query('''
SELECT p.pays, round(((s.nb_personnes*100)/p.population),2) AS prop_personnes_sousalim
FROM population p, sous_nutrition s
WHERE p.année=2013 AND p.année = s.année AND p.code_pays = s.code_pays
ORDER BY prop_personnes_sousalim DESC
LIMIT 10;
''', db)

## Affichage des résultats
print('-'*83)
print('10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte:')
display(R3)

-----------------------------------------------------------------------------------
10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte:


,pays,prop_personnes_sousalim
0,Haïti,50.40
1,République centrafricaine,49.83
2,Zambie,49.52
3,République populaire démocratique de Corée,41.37
4,Libéria,39.59
5,Zimbabwe,39.58
6,Tchad,37.43
7,Madagascar,35.33
8,Ouganda,33.53
9,Rwanda,33.12


[Retour au sommaire](#sommaire)

# <font color=blue>Requête 4 <a id='3.4'> </a>
    10 produits pour lesquels le ratio Autres utilisations/Disponibilité intérieure est le plus élevé.

In [42]:
R4 = pd.read_sql_query('''
SELECT produit, sum(autres_utilisations) AS Autre_utilisation, sum(dispo_int) AS Dispo_int,
round(((sum(autres_utilisations) / sum(dispo_int))*100),1) AS ratio
FROM equilibre_prod
WHERE année = 2013
GROUP BY produit
ORDER BY ratio DESC
LIMIT 10;
''', db)

## Affichage des résultats
print('-'*110)
print('Liste des 10 produits pour lesquels le ratio Autres utilisations/Disponibilité intérieure est le plus élevé.:')
display(R4)

--------------------------------------------------------------------------------------------------------------
Liste des 10 produits pour lesquels le ratio Autres utilisations/Disponibilité intérieure est le plus élevé.:


,produit,Autre_utilisation,Dispo_int,ratio
0,"Alcool, non Comestible",21769.0,21741,100.1
1,Huil Plantes Oleif Autr,4733.0,6271,75.5
2,Huile de Palmistes,4625.0,6570,70.4
3,Huile de Palme,35191.0,50409,69.8
4,Girofles,84.0,130,64.6
5,Huile de Colza&Moutarde,12909.0,23416,55.1
6,Graisses Animales Crue,10792.0,22951,47.0
7,Huiles de Poissons,371.0,828,44.8
8,Huile de Soja,17463.0,41961,41.6
9,Plantes Aquatiques,8934.0,23554,37.9


[Retour au sommaire](#sommaire)

# <font color=blue>Analyse des autres utilisations non alimentaires <a id='3.5'> </a>

>Utilisation non alimentaires des produits avec une forte proportion d’autres utilisations par rapport à la
disponibilité intérieure :

- Alcool non alimentaire (principalement éthanol) : utilisé comme carburant, anti-sceptique, solvant
- Huiles de plantes oléifères : utilisées dans les produits cosmétiques (beurre de karité, huile de ricin), ou
comme additifs de produits industriels tels que peintures, mastics (huile de lin)
- Huile de palme (issue de la pulpe qui entoure la graine) et huile de palmiste (issue de la graine) :
utilisées comme additifs dans les préparations alimentaires industrielles, également dans les savons, les
crèmes hydratantes ou les bougies.
- Girofles : en majorité utilisées dans la fabrication de cigarettes indonésiennes.
- Huile de colza : utilisée comme biocarburant.
- Graisses animales : utilisées dans la fabrication de certains savons.
- Huile de soja : utilisée comme biocarburant.

[Retour au sommaire](#sommaire)